In [ ]:
#EXERCISE 1
#Step 1
import mysql.connector as connector
from mysql.connector import Error
from mysql.connector.pooling import MySQLConnectionPool


# Database configuration
dbconfig = {
    "database": "llemondb",
    "user": "root",
    "password": "@Uz0Ch1**1000",
    "host": "localhost"
    
}

# Step 2
try:
    # Create a pool with 2 connections
    pool_a = MySQLConnectionPool(pool_name="pool_a", pool_size=2, **dbconfig)
    print("Connection pool created successfully!")

    # Get a connection from the pool
    connection = pool_a.get_connection()
    cursor = connection.cursor()

except Error as e:
    print("Error while connecting to database:", e)



Connection pool created successfully!


In [ ]:
get_order_detail = """
DELIMITER $$

CREATE PROCEDURE GetOrderDetail(IN customerId INT)
BEGIN
    PREPARE stmt FROM 'SELECT order_id, quantity, subtotal FROM OrderDetails
                        JOIN Orders USING(order_id)
                        WHERE customer_id = ?';
    SET @cid = customerId;
    EXECUTE stmt USING @cid;
    DEALLOCATE PREPARE stmt;
END $$

DELIMITER ;
"""

for stmt in get_order_detail.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'GetOrderDetail' created successfully")


In [ ]:
#Creating Stored Procedure GetMaxQuantity
create_procedure = """
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(quantity) AS Max_Ordered_Quantity
    FROM OrderDetails;
END
"""

cursor.execute(create_procedure)
print("Stored Procedure 'GetMaxQuantity' created successfully.")


In [ ]:
#Creating Cancel Order
cancel_order = """
DELIMITER $$

CREATE PROCEDURE CancelOrder(IN p_order_id INT)
BEGIN
    DELETE FROM Orders WHERE order_id = p_order_id;
    SELECT CONCAT('Order ID ', p_order_id, ' has been cancelled.') AS Confirmation;
END $$

DELIMITER ;
"""

for stmt in cancel_order.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'CancelOrder' created successfully")


In [ ]:
#Creating Check Booking
check_booking = """
DELIMITER $$

CREATE PROCEDURE CheckBooking(
    IN p_booking_date DATE,
    IN p_table_number INT
)
BEGIN
    DECLARE table_status VARCHAR(50);

    IF EXISTS (
        SELECT 1 FROM Bookings
        WHERE booking_date = p_booking_date
        AND table_number = p_table_number
    ) THEN
        SET table_status = 'Booked';
    ELSE
        SET table_status = 'Available';
    END IF;

    SELECT table_status AS TableStatus;
END $$

DELIMITER ;
"""

for stmt in check_booking.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'CheckBooking' created successfully")


In [ ]:
#Create the AddBooking Stored Procedure ---
add_booking = """
DELIMITER $$

CREATE PROCEDURE AddBooking(
    IN p_booking_id INT,
    IN p_customer_id INT,
    IN p_booking_date DATE,
    IN p_table_number INT
)
BEGIN
    INSERT INTO bookings (booking_id, customer_id, booking_date, table_number)
    VALUES (p_booking_id, p_customer_id, p_booking_date, p_table_number);

    SELECT 'New booking record added successfully.' AS Confirmation;
END $$

DELIMITER ;
"""

#Execute procedure creation ---
for stmt in add_booking.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'AddBooking' created successfully.\n")


In [ ]:
# --- Create the UpdateBooking Stored Procedure ---
update_booking = """
DELIMITER $$

CREATE PROCEDURE UpdateBooking(
    IN p_booking_id INT,
    IN p_booking_date DATE
)
BEGIN
    UPDATE bookings
    SET booking_date = p_booking_date
    WHERE booking_id = p_booking_id;

    SELECT CONCAT('Booking ID ', p_booking_id, ' has been updated to ', p_booking_date) AS Confirmation;
END $$

DELIMITER ;
"""

# --- Execute procedure creation ---
for stmt in update_booking.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'UpdateBooking' created successfully.\n")

In [ ]:
#Creating Add Valid Booking
add_valid_booking = """
DELIMITER $$

CREATE PROCEDURE AddValidBooking(
    IN p_booking_date DATE,
    IN p_table_number INT
)
BEGIN
    DECLARE booking_exists INT DEFAULT 0;
    START TRANSACTION;
    SELECT COUNT(*) INTO booking_exists
    FROM Bookings
    WHERE booking_date = p_booking_date
    AND table_number = p_table_number;

    IF booking_exists > 0 THEN
        ROLLBACK;
        SELECT 'Booking declined – table already booked.' AS Message;
    ELSE
        INSERT INTO Bookings (customer_id, staff_id, booking_date, table_number)
        VALUES (1, 1, p_booking_date, p_table_number);
        COMMIT;
        SELECT 'Booking confirmed – successfully added.' AS Message;
    END IF;
END $$

DELIMITER ;
"""

for stmt in add_valid_booking.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'AddValidBooking' created successfully")


In [ ]:
#Create Cancel Booking Procedure
cancel_booking = """
DELIMITER $$
CREATE PROCEDURE CancelBooking(
    IN p_booking_id INT
)
BEGIN
    DELETE FROM Bookings WHERE booking_id = p_booking_id;
    SELECT CONCAT('Booking ID ', p_booking_id, ' cancelled successfully.') AS Confirmation;
END $$

DELIMITER ;
"""
for stmt in cancel_booking.split(';'):
    if stmt.strip():
        cursor.execute(stmt)
connection.commit()
print("Stored Procedure 'CancelBooking' created successfully")


In [ ]:
#CALLING PROCEDURES


#GetMaxQuantity
print("Calling GetMaxQuantity")
cursor.callproc('GetMaxQuantity')
for result in cursor.stored_results():
    print(result.fetchall())

# Call the CancelOrder stored procedure
order_id = 3
cursor.callproc('CancelOrder', [order_id])
# Fetch and display the confirmation message returned by the procedure
for result in cursor.stored_results():
    print(result.fetchall())



#CheckBooking
print("Checking...")
cursor.callproc('CheckBooking', ['2025-10-12', 5])
for result in cursor.stored_results():
    print(result.fetchall())


#Add Booking
print("Adding Booking...")
# --- Call the AddBooking Stored Procedure ---
booking_data = (101, 3, '2025-10-20', 7)
print(f"Calling AddBooking{booking_data}:")

cursor.callproc('AddBooking', booking_data)
for result in cursor.stored_results():
    print(result.fetchall())

#UpdateBooking
print("Updating Booking...")
update_data = (101, '2025-10-25')
cursor.callproc('UpdateBooking', [*update_data])
for result in cursor.stored_results():
    print(result.fetchall())

#AddValidBooking
cursor.callproc('AddValidBooking', ['2025-10-15', 3])
for result in cursor.stored_results():
    print(result.fetchall())


#CancelBooking
cursor.callproc('CancelBooking', [5])
for result in cursor.stored_results():
    print(result.fetchall())
